In [1]:
%env RERUN_NOTEBOOK_ASSET=serve-local

env: RERUN_NOTEBOOK_ASSET=serve-local


In [2]:
import ipywidgets as widgets
from IPython.display import display
import datetime
import math

import rerun as rr
from rerun.notebook import Viewer, ViewerCallbacks, SelectionItem, RecordingStream
from rerun.dataframe import Recording

DEV ENVIRONMENT DETECTED! Re-importing rerun from: /Users/gijsd/rerun/rerun/rerun_py/rerun_sdk


Serving rerun notebook assets at http://127.0.0.1:59168
import failed:  cannot import name 'StorageNodeClient' from 'rerun_bindings' (/Users/gijsd/rerun/rerun/rerun_py/rerun_bindings/__init__.py)


In [3]:
RAW_RECORDING_PATH = "/Users/gijsd/rerun/pusht_image_raw.rrd"
raw_recording = rr.dataframe.load_recording(RAW_RECORDING_PATH)
print("Loaded recording:", raw_recording.application_id(), raw_recording.recording_id())

annotated_recording = rr.new_recording(
    application_id=raw_recording.application_id(), recording_id=raw_recording.recording_id()
)
rr.log_file_from_path(file_path=RAW_RECORDING_PATH, recording=annotated_recording)

Loaded recording: "/Users/gijsd/rerun/redap-dev-tools/pusht_image" episode_105


[2025-02-28T11:17:42Z DEBUG re_log_encoding::decoder] Reached end of stream, iterator complete
[2025-02-28T11:17:42Z DEBUG re_chunk::batcher] creating new chunk batcher config=ChunkBatcherConfig { flush_tick: 8ms, flush_num_bytes: 1048576, flush_num_rows: 18446744073709551615, chunk_max_rows_if_unsorted: 256, max_commands_in_flight: None, max_chunks_in_flight: None, hooks: BatcherHooks { on_insert: None, on_release: Some(ArrowRecordBatchReleaseCallback("0x2726e020260")) } }
[2025-02-28T11:17:42Z DEBUG re_sdk::recording_stream] setting recording info app_id="/Users/gijsd/rerun/redap-dev-tools/pusht_image" rec_id=episode_105
[2025-02-28T11:17:42Z INFO  re_data_loader::load_file] Loading "/Users/gijsd/rerun/pusht_image_raw.rrd"…
[2025-02-28T11:17:42Z DEBUG re_data_loader::loader_rrd] Loading rrd data from filesystem… filepath="/Users/gijsd/rerun/pusht_image_raw.rrd" loader="rerun.data_loaders.Rrd"
[2025-02-28T11:17:42Z DEBUG re_data_loader::load_file] compatible loader found loader="rerun

In [4]:
class RecordingMarker(ViewerCallbacks):
    def __init__(self, viewer: Viewer, annotated_recording: RecordingStream, raw_recording: Recording):
        self.viewer = viewer
        self.annotated_recording = annotated_recording
        self.raw_recording = raw_recording
        self.timeline = "log_time"
        self.time = 0
        self.entity_path = ""

        # Create widgets
        self.timestamp_button = widgets.Button(
            description="Add Timestamp", button_style="primary", tooltip="Mark current position in the recording"
        )

        self.comment_text = widgets.Text(
            placeholder="Optional comment", description="Comment:", style={"description_width": "initial"}
        )

        self.markers_output = widgets.Output(
            layout={
                "border": "1px solid #ddd",
                "padding": "10px",
                "margin": "10px 0px",
                "max_height": "300px",
                "overflow": "auto",
                "color": "white",
            }
        )

        # Label for timeline, time, and entity path
        self.time_label = widgets.Label(value=self._get_time_label_text())

        # Replace status output with status label
        self.status_label = widgets.Label(value="")

        # Add image widget
        self.image_widget = widgets.Image(
            format="png", width=400, height=300, layout=widgets.Layout(border="1px solid #ddd", margin="10px 0px")
        )

        # Add a NumPy widget container (Output widget where numpy content can be displayed)
        self.numpy_output = widgets.Output(
            layout={
                "border": "1px solid #ddd",
                "padding": "10px",
                "margin": "10px 0px",
                "min_height": "200px",
            }
        )

        self.timestamp_button.on_click(self.add_marker)
        self.widget = widgets.VBox(
            [
                self.time_label,
                widgets.HBox([self.timestamp_button, self.comment_text]),
                self.markers_output,
                self.status_label,
                self.image_widget,
                self.numpy_output,
            ],
            layout=widgets.Layout(
                background_color="white",
                padding="10px",
            ),
        )

    def _get_time_label_text(self):
        """Helper to construct the text for the time label including entity path."""
        entity_info = f" | Selected: {self.entity_path}" if self.entity_path else ""
        return f"{self.timeline}: {self.time}{entity_info}"

    def on_time_update(self, time: float):
        self.time = int(time)
        self.update_timeline()

    def on_timeline_change(self, timeline: str, time: float):
        self.timeline = timeline
        self.time = int(time)
        self.update_timeline()

    def update_timeline(self):
        # Update the Label widget with time and entity path
        self.time_label.value = self._get_time_label_text()

    def on_selection_change(self, selection: list[SelectionItem]):
        for s in selection:
            if s.kind == "entity_path":
                self.entity_path = s.entity_path
                self.update_timeline()

    def add_marker(self, button):
        """Add a new timestamp marker with optional comment."""
        comment = self.comment_text.value.strip()

        rr.set_time_sequence(timeline=self.timeline, sequence=self.time, recording=self.annotated_recording)
        rr.log(self.entity_path, rr.AnyValues(marker=(), comment=comment), recording=self.annotated_recording)

        # Update status label instead of using output widget
        self.status_label.value = f"✓ Marker added at {self.time} ({self.timeline})"
        self.comment_text.value = ""

    def update_image(self, image_data):
        """Update the image widget with new image data.

        Args:
            image_data: NumPy array containing image data
        """
        if image_data is not None:
            # Convert numpy array to bytes for the Image widget
            import io
            from PIL import Image
            import numpy as np

            # Ensure image data is in the right format (uint8)
            if image_data.dtype != np.uint8:
                image_data = (image_data * 255).astype(np.uint8)

            # Create PIL Image and convert to bytes
            pil_image = Image.fromarray(image_data)
            buffer = io.BytesIO()
            pil_image.save(buffer, format="PNG")
            self.image_widget.value = buffer.getvalue()

    def update_numpy_widget(self, numpy_content):
        """
        Display NumPy content in the numpy output widget.

        Args:

            numpy_content: The numpy array or data to display
        """
        with self.numpy_output:
            self.numpy_output.clear_output()
            import matplotlib.pyplot as plt

            # Display numpy data or visualization
            plt.figure(figsize=(6, 4))
            if len(numpy_content.shape) == 1:
                plt.plot(numpy_content)
            else:
                plt.imshow(numpy_content)
            plt.colorbar()
            plt.tight_layout()
            plt.show()

    def display(self):
        """Display the widget."""
        display(self.widget)

In [5]:
viewer = Viewer(
    width=1280,
    height=720,
    recording=annotated_recording,
)
marker_callback = RecordingMarker(viewer, annotated_recording, raw_recording)
viewer.register_callbacks(marker_callback)

display(viewer)
marker_callback.display()

[2025-02-28T11:17:42Z DEBUG re_sdk::recording_stream] setting recording info app_id="/Users/gijsd/rerun/redap-dev-tools/pusht_image" rec_id=episode_105


Viewer()

In [7]:
raw_recording.view(index="frame_index", contents="/observation.image").select().read_all()

pyarrow.Table
frame_index: int64
/observation.image:Blob: list<item: list<item: uint8 not null>>
  child 0, item: list<item: uint8 not null>
      child 0, item: uint8 not null
/observation.image:MediaType: list<item: string>
  child 0, item: string
----
frame_index: [[0],[1],...,[73],[74]]
/observation.image:Blob: [[[[137,80,78,71,13,...,68,174,66,96,130]]],[[[137,80,78,71,13,...,68,174,66,96,130]]],...,[[[137,80,78,71,13,...,68,174,66,96,130]]],[[[137,80,78,71,13,...,68,174,66,96,130]]]]
/observation.image:MediaType: [[["image/png"]],[["image/png"]],...,[["image/png"]],[["image/png"]]]